# Keras Overview

An overview of using Keras 3 for building an autoencoder and using it for anomaly detection.

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('jax','jax'),
    ('keras', 'keras', '3.6.0'),
    ('tensorflow', 'tensorflow'),
    ('pydot', 'pydot')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Graphviz Install

Plotting the [model structure with Keras](https://keras.io/api/utils/model_plotting_utils/) uses [Graphviz](https://graphviz.org/download/).

This code check for Graphviz and if missing installs it.

In [4]:
check = !dot -V
if check[0].startswith('dot'):
    print(f'Graphviz installed with version: {check[0]}')
else:
    print('Installing Graphviz...')
    install = !sudo apt-get install graphviz --assume-yes
    print('Completed')

Graphviz installed with version: dot - graphviz version 2.43.0 (0)


### API Enablement

In [5]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [6]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [7]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [8]:
REGION = 'us-central1'
SERIES = 'frameworks-keras'
EXPERIMENT = 'overview'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

# make this the BigQuery Project / Dataset / Table prefix to store results
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE = EXPERIMENT
BQ_REGION = REGION[0:2] # use a multi region

Packages

In [9]:
import os#, json, time, glob

#import numpy as np

# import keras, set backend prior to first import
os.environ['KERAS_BACKEND'] = 'jax'
import keras
import tensorflow as tf

# Vertex AI
from google.cloud import aiplatform

# BigQuery
from google.cloud import bigquery

2024-12-16 11:50:19.538154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734349819.581804 2028495 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734349819.596049 2028495 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
aiplatform.__version__

'1.71.0'

In [11]:
keras.__version__

'3.6.0'

In [12]:
tf.__version__

'2.18.0'

Clients

In [13]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

---
## Review Source Data

This is a BigQuery public table of 284,807 credit card transactions classified as fradulant or normal in the column `Class`.
- The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).
- Read mode about BigQuery public datasets [here](https://cloud.google.com/bigquery/public-data)

In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction
 

### Review BigQuery table:

In [14]:
source_data = bq.query(f'SELECT * FROM `{BQ_SOURCE}` LIMIT 5').to_dataframe()
source_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748.0,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0,0
1,27074.0,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.0,0
2,28292.0,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.0,0
3,28488.0,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.0,0
4,31392.0,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.0,0


In [15]:
source_data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       Int64
dtype: object

---
## Prepare Data Source

The data preparation includes adding splits for machine learning with a column named `splits` with 80% for training (`TRAIN`), 10% for validation (`VALIDATE`) and 10% for testing (`TEST`).  Additionally, a unique identifier was added to each transaction, `transaction_id`. 

### Create/Recall Dataset

In [16]:
dataset = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
dataset.location = BQ_REGION
bq_dataset = bq.create_dataset(dataset, exists_ok = True)

### Create/Recall Table With Preparation For ML

Copy the data from the source while adding columns:
- `transaction_id` as a unique identify for the row
    - Use the `GENERATE_UUID()` function
- `splits` column to randomly assign rows to 'TRAIN", "VALIDATE" and "TEST" groups
    - stratified sampling within the levels of `class` by first assigning row numbers within the levels of `class` then using the with a CASE statment to assign the `splits` level.

In [17]:
job = bq.query(f"""
#CREATE OR REPLACE TABLE
CREATE TABLE IF NOT EXISTS 
    `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
WITH
    add_id AS (
        SELECT *,
            GENERATE_UUID() transaction_id,
            ROW_NUMBER() OVER (PARTITION BY class ORDER BY RAND()) as rn
            FROM `{BQ_SOURCE}`
    )
SELECT * EXCEPT(rn),
    CASE 
        WHEN rn <= 0.8 * COUNT(*) OVER (PARTITION BY class) THEN 'TRAIN'
        WHEN rn <= 0.9 * COUNT(*) OVER (PARTITION BY class) THEN 'VALIDATE'
        ELSE 'TEST'
    END AS splits
FROM add_id
""")
job.result()
(job.ended-job.started).total_seconds()

0.499

In [18]:
raw_sample = bq.query(f'SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` LIMIT 5').to_dataframe()
raw_sample

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,43330.0,-1.510308,0.780684,2.085747,3.123133,0.589557,1.301681,-1.300991,-2.345333,0.148722,...,-0.341324,-0.370869,-0.201199,0.423964,0.210998,-0.058616,0.0,0,34a1f550-443c-45f0-858d-94cf0baebfe8,TEST
1,141955.0,-1.585532,1.240411,-0.558349,-0.841657,0.397502,-1.532723,0.388818,0.575896,-0.278370,...,-0.249493,0.001462,-0.045791,-0.110667,0.305249,0.015271,0.0,0,a2dfc3ff-aa20-4742-bb1c-c6f67440e34a,TEST
2,58813.0,-2.524342,-0.313784,2.304085,3.043016,5.495686,-2.600347,-3.779679,-1.042154,-1.014503,...,-6.185491,0.440080,-1.409641,-0.209465,-0.111065,0.116788,0.0,0,d7910b3a-fc5e-417c-8b61-5516cc4dc981,TEST
3,139716.0,-2.006121,2.048709,-0.963971,-1.046216,-0.118273,-1.400030,0.401296,0.728337,0.095944,...,0.059168,-0.113542,0.066162,-0.143905,-0.218363,-0.091904,0.0,0,4de5ecd4-7c82-40dd-a52a-4b5e8b1e8ba3,TEST
4,59183.0,1.175167,-0.087900,0.278759,-0.279214,-0.166236,0.058772,-0.232681,0.192241,-0.014614,...,0.165047,-0.253526,-0.016523,0.890504,-0.059400,-0.013996,0.0,0,ec122bde-5780-4223-9e35-7f26de453512,TEST


### Review the number of records for each level of `Class` for each of the data splits:

In [19]:
bq.query(f"""
SELECT splits, class,
    count(*) as count,
    ROUND(count(*) * 100.0 / SUM(count(*)) OVER (PARTITION BY class), 2) AS percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
GROUP BY splits, class
""").to_dataframe()

,splits,class,count,percentage
0,TEST,1,50,10.16
1,TRAIN,1,393,79.88
2,VALIDATE,1,49,9.96
3,TEST,0,28432,10.00
4,TRAIN,0,227452,80.00
5,VALIDATE,0,28431,10.00


## Training - Local, Simple

---
Data, Readers, Prep Function, and Normalizers

In [20]:
train_ds = bq.query(f"SELECT * EXCEPT(splits) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'TRAIN'").to_dataframe()
test_ds = bq.query(f"SELECT * EXCEPT(splits) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'TEST'").to_dataframe()
validate_ds = bq.query(f"SELECT * EXCEPT(splits) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'VALIDATE'").to_dataframe()

In [21]:
train_ds.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class', 'transaction_id'],
      dtype='object')

In [22]:
# Define column categories
var_class = ['Class']
var_omit = ['transaction_id']
var_numeric = [x for x in train_ds.columns.tolist() if x not in var_class + var_omit]

In [23]:
def reader(ds):
    return tf.data.Dataset.from_tensor_slices(dict(ds))

In [24]:
train_read = reader(train_ds)
validate_read = reader(validate_ds)
test_read = reader(test_ds)

2024-12-16 11:51:08.595033: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [25]:
batch = next(iter(train_read.batch(10).take(1)))
batch.keys()

dict_keys(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class', 'transaction_id'])

In [26]:
batch['Time']

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([ 83533., 113822.,  39200.,  61781., 148437., 123042., 127907.,
         1888., 117742., 163935.])>

In [27]:
def prep_batch(source):
    for k in var_omit + var_class:
        source.pop(k, None)
    numeric_values = tf.stack([source[col] for col in var_numeric], axis=-1)
    return numeric_values, numeric_values

In [28]:
batch, batch = next(iter(train_read.map(prep_batch).batch(10).take(1)))

In [29]:
batch.shape

TensorShape([10, 30])

In [30]:
batch[0]

<tf.Tensor: shape=(30,), dtype=float64, numpy=
array([ 8.35330000e+04,  1.12552389e+00,  1.89853946e-01,  1.50865969e+00,
        2.71993690e+00, -8.82341741e-01,  1.28390258e-01, -6.22650925e-01,
        2.19444433e-01,  2.55370444e-01,  5.21744641e-01, -1.11755689e+00,
       -3.42581768e-01, -9.18035487e-01, -2.24182825e-01, -1.98108066e-01,
        6.65628330e-01, -3.60429717e-01, -8.31641571e-02, -8.35711759e-01,
       -2.25172417e-01, -9.75838741e-02, -1.72683342e-01,  5.75558670e-02,
        3.60859312e-01,  2.65443187e-01, -4.04174990e-02,  3.66861302e-02,
        3.36852774e-02,  0.00000000e+00])>

In [31]:
# Normalization
normalizer = keras.layers.Normalization(axis=-1, name='normalization')

# Adapt the normalizer 
numeric_feature_reader = train_read.map(prep_batch).batch(1000).prefetch(tf.data.AUTOTUNE)
normalizer.adapt(numeric_feature_reader.map(lambda x, _: x))

# Denormalization using normalizer parameters
denormalizer = keras.layers.Normalization(axis = -1, name = 'denormalize', invert = True, mean = normalizer.mean, variance = normalizer.variance)

---
Autoencoder #1 - Bones

In [32]:
autoencoder_input = keras.Input(shape = (len(var_numeric),), name = "autoencoder_input")
#normalized_input = normalizer(autoencoder_input)
encoder = keras.layers.Dense(16, activation='relu', name='enc_dense1', kernel_regularizer = keras.regularizers.l2(0.001))(autoencoder_input)
encoder = keras.layers.Dropout(0.4, name='enc_dropout1')(encoder)
encoder = keras.layers.Dense(8, activation='relu', name='enc_dense2', kernel_regularizer = keras.regularizers.l2(0.001))(encoder)
encoder = keras.layers.Dropout(0.4, name='enc_dropout2')(encoder)
encoder = keras.layers.Dense(4, activation='relu', name='enc_dense3', kernel_regularizer = keras.regularizers.l2(0.001))(encoder)
decoder = keras.layers.Dense(8, activation='relu', name='dec_dense1', kernel_regularizer = keras.regularizers.l2(0.001))(encoder)
decoder = keras.layers.Dropout(0.4, name='dec_dropout1')(decoder)
decoder = keras.layers.Dense(16, activation='relu', name='dec_dense2', kernel_regularizer = keras.regularizers.l2(0.001))(decoder)
decoder = keras.layers.Dropout(0.4, name='dec_dropout2')(decoder)
decoder = keras.layers.Dense(len(var_numeric), activation='linear', name='dec_dense3')(decoder)
reconstructed = decoder
#denormalizer(decoder)

In [33]:
autoencoder = keras.Model(autoencoder_input, reconstructed, name = 'autoencoder')

In [34]:
import jax.numpy as jnp
def custom_loss(y_true, y_pred):
    return jnp.mean(jnp.abs(y_true - y_pred), axis=-1)

In [35]:
autoencoder.compile(
    optimizer = keras.optimizers.Adam(learning_rate = 0.0005),
    loss = custom_loss, #keras.losses.MeanAbsoluteError(),
    metrics = [
        keras.metrics.RootMeanSquaredError(name = 'rmse'),
        keras.metrics.MeanSquaredError(name = 'mse'),
        keras.metrics.MeanAbsoluteError(name = 'mae'),
        keras.metrics.MeanSquaredLogarithmicError(name = 'msle')
    ]
)

In [36]:
autoencoder.summary()

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ autoencoder_input (InputLayer)  │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense1 (Dense)              │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dropout1 (Dropout)          │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense2 (Dense)              │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dropout2 (Dropout)          │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense3 (Dense)              │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dense1 (Dense)              │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dropout1 (Dropout)          │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dense2 (Dense)              │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dropout2 (Dropout)          │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dense3 (Dense)              │ (None, 30)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,362 (5.32 KB)

 Trainable params: 1,362 (5.32 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Data preparation for training
def reshape_data(x, y):
    return tf.reshape(x, (-1, len(var_numeric))), tf.reshape(y, (-1, len(var_numeric)))


# Data preparation for training
train_dataset = train_read \
    .shuffle(buffer_size = len(train_ds), reshuffle_each_iteration = True) \
    .map(prep_batch) \
    .map(lambda x, _: (normalizer(x), normalizer(x))) \
    .batch(100) \
    .map(reshape_data) \
    .prefetch(tf.data.AUTOTUNE)
val_dataset = validate_read \
    .map(prep_batch) \
    .map(lambda x, _: (normalizer(x), normalizer(x))) \
    .batch(100) \
    .map(reshape_data) \
    .prefetch(tf.data.AUTOTUNE)

In [38]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor = "val_loss",  # Monitor validation loss
    patience = 5,          # Number of epochs with no improvement before stopping
    restore_best_weights = True, # using the model that generalized best to the validation set, not the overfitted model from the last epoch
)

In [39]:
history = autoencoder.fit(
    train_dataset,
    epochs = 10,
    validation_data = val_dataset,
    callbacks = [early_stopping]
)

Epoch 1/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 37s 14ms/step - loss: 0.6896 - mae: 0.6607 - mse: 1.0094 - msle: 0.1456 - rmse: 1.0047 - val_loss: 0.6392 - val_mae: 0.6355 - val_mse: 0.9665 - val_msle: 0.1321 - val_rmse: 0.9831
Epoch 2/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 43s 16ms/step - loss: 0.6421 - mae: 0.6385 - mse: 0.9800 - msle: 0.1334 - rmse: 0.9899 - val_loss: 0.6286 - val_mae: 0.6250 - val_mse: 0.9526 - val_msle: 0.1288 - val_rmse: 0.9760
Epoch 3/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 0.6366 - mae: 0.6330 - mse: 0.9671 - msle: 0.1312 - rmse: 0.9834 - val_loss: 0.6277 - val_mae: 0.6240 - val_mse: 0.9518 - val_msle: 0.1283 - val_rmse: 0.9756
Epoch 4/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 0.6372 - mae: 0.6335 - mse: 0.9822 - msle: 0.1312 - rmse: 0.9910 - val_loss: 0.6276 - val_mae: 0.6239 - val_mse: 0.9521 - val_msle: 0.1279 - val_rmse: 0.9757
Epoch 5/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 0.6351 - mae: 0.6314 - mse: 0.9654 - msle: 0.130

In [40]:
test_instance = next(iter(test_read.map(prep_batch).batch(2).take(1)))[0] # get first element of tuple for prediction
test_instance

<tf.Tensor: shape=(2, 30), dtype=float64, numpy=
array([[ 4.33300000e+04, -1.51030791e+00,  7.80683602e-01,
         2.08574736e+00,  3.12313283e+00,  5.89556504e-01,
         1.30168094e+00, -1.30099101e+00, -2.34533311e+00,
         1.48722209e-01,  1.06300824e+00, -1.88035726e+00,
         3.10971711e-01,  9.54298638e-01, -1.10646280e+00,
        -7.86015840e-01, -5.55324333e-01,  3.35236557e-01,
         1.85637688e-01,  1.54380629e+00, -6.44864027e-01,
         2.22981820e+00, -4.60620924e-02, -3.41323962e-01,
        -3.70869344e-01, -2.01199298e-01,  4.23963954e-01,
         2.10998182e-01, -5.86163935e-02,  0.00000000e+00],
       [ 1.41955000e+05, -1.58553155e+00,  1.24041103e+00,
        -5.58349101e-01, -8.41657245e-01,  3.97502038e-01,
        -1.53272269e+00,  3.88817678e-01,  5.75896042e-01,
        -2.78369582e-01, -8.56928209e-01, -1.11892547e+00,
         7.37223588e-01,  4.79262560e-01,  7.95002104e-01,
        -3.96344845e-01, -3.62277829e-01,  4.99583980e-02,
      

In [41]:
denormalizer(autoencoder.predict(normalizer(test_instance)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


Array([[ 6.33595391e+04, -4.20057744e-01,  1.48006648e-01,
         7.25894570e-01,  4.94559184e-02, -2.20891267e-01,
        -1.69500351e-01, -2.02774536e-02,  7.83200637e-02,
        -1.37183920e-01, -1.25701457e-01,  5.48960567e-02,
         1.28669426e-01,  6.18027465e-04,  4.10104282e-02,
         2.35474437e-01,  7.26926848e-02, -2.49771904e-02,
        -4.08719443e-02, -8.73417128e-03, -2.70890109e-02,
        -5.59706204e-02, -8.53525996e-02, -4.24885936e-02,
         5.62276468e-02,  1.29422456e-01, -5.70486486e-02,
         1.27739357e-02,  2.31126305e-02,  2.23041306e+01],
       [ 1.28268203e+05,  5.86209238e-01,  8.75001214e-03,
        -5.95429778e-01, -1.83047622e-01,  1.87409014e-01,
        -4.00892466e-01,  1.22237094e-01, -4.07874696e-02,
         5.19790277e-02, -5.63699529e-02, -1.72968641e-01,
         1.42069593e-01, -1.51216555e-02,  5.11047170e-02,
        -1.40069112e-01,  8.07579160e-02, -1.35525376e-01,
         5.06579317e-02,  2.88527925e-03, -1.02698289e-

---
Combine the normalizer, model, and denormalizer in new model
- a neat part about keras is that models can be layers in new models!

In [48]:
type(normalizer), type(autoencoder), type(denormalizer)

(keras.src.layers.preprocessing.normalization.Normalization,
 keras.src.models.functional.Functional,
 keras.src.layers.preprocessing.normalization.Normalization)

In [49]:
# Create a new sequential model
stacked_model = keras.Sequential([
    normalizer,
    autoencoder,
    denormalizer
], name='stacked_autoencoder')

In [50]:
test_instance = next(iter(test_read.map(prep_batch).batch(3).take(1)))[0]
stacked_model.predict(test_instance)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


array([[ 6.33595391e+04, -4.20057744e-01,  1.48006648e-01,
         7.25894630e-01,  4.94559184e-02, -2.20891267e-01,
        -1.69500351e-01, -2.02774536e-02,  7.83200562e-02,
        -1.37183905e-01, -1.25701457e-01,  5.48960604e-02,
         1.28669426e-01,  6.18027523e-04,  4.10104282e-02,
         2.35474437e-01,  7.26926848e-02, -2.49771904e-02,
        -4.08719443e-02, -8.73417035e-03, -2.70890091e-02,
        -5.59706204e-02, -8.53525996e-02, -4.24885936e-02,
         5.62276468e-02,  1.29422456e-01, -5.70486486e-02,
         1.27739357e-02,  2.31126305e-02,  2.23041344e+01],
       [ 1.28268211e+05,  5.86209238e-01,  8.74999911e-03,
        -5.95429778e-01, -1.83047622e-01,  1.87409043e-01,
        -4.00892466e-01,  1.22237094e-01, -4.07874696e-02,
         5.19790091e-02, -5.63699529e-02, -1.72968641e-01,
         1.42069593e-01, -1.51216593e-02,  5.11047170e-02,
        -1.40069112e-01,  8.07579234e-02, -1.35525376e-01,
         5.06579317e-02,  2.88527925e-03, -1.02698289e-

In [51]:
reversed_test_instance = tf.reverse(test_instance, axis=[-1])
stacked_model.predict(reversed_test_instance)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[ 7.74654297e+04, -2.52681077e-01,  1.27747312e-01,
         4.55224365e-01, -1.49913561e-02, -1.25400349e-01,
        -2.17116788e-01,  1.65133644e-02,  5.37722781e-02,
        -1.01522394e-01, -1.16729915e-01,  2.30901269e-03,
         1.27710238e-01, -5.28460732e-05,  4.22737598e-02,
         1.49507880e-01,  7.74287209e-02, -4.99096513e-02,
        -1.72923040e-02, -6.21818565e-03, -4.25109901e-02,
        -4.24873605e-02, -4.25087623e-02, -2.63656247e-02,
         4.75233309e-02,  6.80980384e-02, -5.20702787e-02,
         8.53120722e-03,  1.54721215e-02,  2.14464283e+01],
       [ 1.40690516e+05,  9.59434211e-01, -5.31703234e-02,
        -9.35072124e-01, -1.71330422e-01,  2.51100212e-01,
        -4.62747961e-01,  1.38657197e-01, -7.41311908e-02,
         1.04735710e-01, -1.82096995e-02, -2.14068696e-01,
         1.58013433e-01, -2.93077156e-02,  6.73388019e-02,
        -2.04518288e-01,  6.93024769e-02, -1.59243271e-01,
         5.63917458e-02,  9.98867024e-03, -1.22684337e-

In [52]:
stacked_model.summary()

Model: "stacked_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (3, 30)                │            61 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ autoencoder (Functional)        │ (3, 30)                │         1,362 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ denormalize (Normalization)     │ (3, 30)                │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,423 (5.56 KB)

 Trainable params: 1,362 (5.32 KB)

 Non-trainable params: 61 (244.00 B)

In [53]:
stacked_model.summary(expand_nested=True)

Model: "stacked_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (3, 30)                │            61 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ autoencoder (Functional)        │ (3, 30)                │         1,362 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ autoencoder_input          │ (None, 30)             │             0 │
│ (InputLayer)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ enc_dense1 (Dense)         │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ enc_dropout1 (Dropout)     │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ enc_dense2 (Dense)         │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ enc_dropout2 (Dropout)     │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ enc_dense3 (Dense)         │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dec_dense1 (Dense)         │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dec_dropout1 (Dropout)     │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dec_dense2 (Dense)         │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dec_dropout2 (Dropout)     │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dec_dense3 (Dense)         │ (None, 30)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ denormalize (Normalization)     │ (3, 30)                │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,423 (5.56 KB)

 Trainable params: 1,362 (5.32 KB)

 Non-trainable params: 61 (244.00 B)

---
Named inputs, a dictionary of features
- input dictionary, concatenate features, pass to the stacked model

Why?
- user-friendly!
- prevent training serving skew from feature order causesing serving error


Using an ordered dictionary for the input layers ensures the inputs order as they pass to the concatenation layers is consistent.  The inputs to the model can actually be out of order but this ordering of the input layer will force the consistency needed.

In [54]:
from collections import OrderedDict

input_dict = OrderedDict()
for key in var_numeric:
    input_dict[key] = keras.layers.Input(shape=(1,), name=key)

In [55]:
concatenated_inputs = keras.layers.Concatenate(axis = -1)(list(input_dict.values()))

In [56]:
final_model = keras.Model(inputs = input_dict, outputs = stacked_model(concatenated_inputs))

In [57]:
def prep_batch_dict(source):
    for k in var_omit + var_class:
        source.pop(k, None)

    # Create an ordered dictionary with empty lists for each key
    result = OrderedDict((key, []) for key in var_numeric)

    # Iterate through each column and append the value to the corresponding list
    for key in var_numeric:
        result[key].append(source[key])

    # Convert lists to tensors
    for key in result:
        result[key] = tf.concat(result[key], axis=0)

    return result

In [58]:
test_instance = next(iter(test_read.map(prep_batch_dict).batch(3).take(1)))

In [59]:
final_model.predict(test_instance)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step


array([[ 6.33595391e+04, -4.20057744e-01,  1.48006648e-01,
         7.25894630e-01,  4.94559184e-02, -2.20891267e-01,
        -1.69500351e-01, -2.02774536e-02,  7.83200562e-02,
        -1.37183905e-01, -1.25701457e-01,  5.48960604e-02,
         1.28669426e-01,  6.18027523e-04,  4.10104282e-02,
         2.35474437e-01,  7.26926848e-02, -2.49771904e-02,
        -4.08719443e-02, -8.73417035e-03, -2.70890091e-02,
        -5.59706204e-02, -8.53525996e-02, -4.24885936e-02,
         5.62276468e-02,  1.29422456e-01, -5.70486486e-02,
         1.27739357e-02,  2.31126305e-02,  2.23041344e+01],
       [ 1.28268211e+05,  5.86209238e-01,  8.74999911e-03,
        -5.95429778e-01, -1.83047622e-01,  1.87409043e-01,
        -4.00892466e-01,  1.22237094e-01, -4.07874696e-02,
         5.19790091e-02, -5.63699529e-02, -1.72968641e-01,
         1.42069593e-01, -1.51216593e-02,  5.11047170e-02,
        -1.40069112e-01,  8.07579234e-02, -1.35525376e-01,
         5.06579317e-02,  2.88527925e-03, -1.02698289e-

In [60]:
test_instance

OrderedDict([('Time',
              <tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 43330., 141955.,  58813.])>),
             ('V1',
              <tf.Tensor: shape=(3,), dtype=float64, numpy=array([-1.51030791, -1.58553155, -2.52434169])>),
             ('V2',
              <tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 0.7806836 ,  1.24041103, -0.3137836 ])>),
             ('V3',
              <tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 2.08574736, -0.5583491 ,  2.30408519])>),
             ('V4',
              <tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.12313283, -0.84165725,  3.04301624])>),
             ('V5',
              <tf.Tensor: shape=(3,), dtype=float64, numpy=array([0.5895565 , 0.39750204, 5.49568553])>),
             ('V6',
              <tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 1.30168094, -1.53272269, -2.60034741])>),
             ('V7',
              <tf.Tensor: shape=(3,), dtype=float64, numpy=array([-1.30099101,  0.38881768

In [61]:
final_model.summary(expand_nested = True)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Time (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V4 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V5 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V6 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V7 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V8 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V9 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V10 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V11 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V12 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V13 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V14 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V15 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V16 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V17 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V18 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V19 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V20 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V21 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V22 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V23 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V24 (InputLayer)    │ (None, 1)         │          0 │ -               

 Total params: 1,423 (5.56 KB)

 Trainable params: 1,362 (5.32 KB)

 Non-trainable params: 61 (244.00 B)

---
Customizing the outputs:
    - reconstructed values: normalized, denormalized
    - errors in reconstructed values: normalized, denormalized
    - embedding layer
    - error metrics: MAE, MSE, RMSE    

In [63]:
# Get encoder output
encoder_output = autoencoder.get_layer('enc_dense3').output

In [66]:
# L2 normalize the embedding using Lambda layer with output_shape
normalized_embedding = keras.layers.Lambda(
    lambda x: x / tf.sqrt(tf.reduce_sum(tf.square(x), axis=-1, keepdims=True)),
    output_shape=lambda input_shape: input_shape
)(encoder_output)

In [67]:
# Connect the concatenated inputs to the stacked model
denormalized_output = stacked_model(concatenated_inputs)

In [70]:
# Create outputs for the model
normalized_reconstruction = autoencoder(concatenated_inputs)
denormalized_reconstruction = denormalized_output
# Calculate reconstruction errors using Lambda layers and TensorFlow operations
normalized_reconstruction_error = keras.layers.Lambda(
    lambda x: tf.abs(x[0] - x[1]),
    output_shape=lambda input_shape: input_shape[0]
)([concatenated_inputs, normalized_reconstruction])
denormalized_reconstruction_error = keras.layers.Lambda(
    lambda x: tf.abs(x[0] - x[1]),
    output_shape=lambda input_shape: input_shape[0]
)([denormalizer(concatenated_inputs), denormalized_reconstruction])

In [71]:
# Create the final model with structured output
final_model = keras.Model(
    inputs=input_dict,
    outputs={
        'reconstruction': {
            'normalized': normalized_reconstruction,
            'denormalized': denormalized_reconstruction,
        },
        'reconstruction_errors': {
            'normalized': normalized_reconstruction_error,
            'denormalized': denormalized_reconstruction_error,
        },
        'embedding': {
            'encoding': encoder_output,
            'normalized_encoding': normalized_embedding,
        }
    }
)

In [72]:
test_instance = next(iter(test_read.map(prep_batch_dict).batch(3).take(1)))

In [73]:
final_model.predict(test_instance)

AttributeError: 'str' object has no attribute '_error_repr'

In [75]:
# 1. Prepare a Single Batch with Correct Structure
test_read_dict = test_read.map(prep_batch_dict)
test_batch = next(iter(test_read_dict.batch(3)))  # No need for .take(1)

# Access the dictionary (first element of the tuple)
test_instance_dict = test_batch

# 2. Perform Prediction
predictions = final_model.predict(test_instance_dict)

TypeError: 'NoneType' object is not callable

Notes:
- outputs customized
- more epochs but using the full model
    - or the inner model - how does it update the full
- supervised (maybe another workflow)
- semi-superfixed with pre-train and fine-tune ( 
- specific
    - rename middle layer: embedding
    - output normalized version of this
- export/import example
- for dict input convert test instance to actual dictionary:
    - show inference
    - add extra keys, show inference

---
Issue - Instance Order Matters
- getting the instance columns out of order will lead to mistakes

In [67]:
test_instance = next(iter(test_read.map(prep_batch).batch(1).take(1)))[0] # get first element of tuple for prediction
test_instance

<tf.Tensor: shape=(1, 30), dtype=float64, numpy=
array([[ 4.33300000e+04, -1.51030791e+00,  7.80683602e-01,
         2.08574736e+00,  3.12313283e+00,  5.89556504e-01,
         1.30168094e+00, -1.30099101e+00, -2.34533311e+00,
         1.48722209e-01,  1.06300824e+00, -1.88035726e+00,
         3.10971711e-01,  9.54298638e-01, -1.10646280e+00,
        -7.86015840e-01, -5.55324333e-01,  3.35236557e-01,
         1.85637688e-01,  1.54380629e+00, -6.44864027e-01,
         2.22981820e+00, -4.60620924e-02, -3.41323962e-01,
        -3.70869344e-01, -2.01199298e-01,  4.23963954e-01,
         2.10998182e-01, -5.86163935e-02,  0.00000000e+00]])>

In [68]:
denormalizer(autoencoder.predict(normalizer(test_instance)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


Array([[ 5.91586602e+04, -4.50955182e-01,  1.37564048e-01,
         7.74269998e-01,  6.93160743e-02, -2.52150357e-01,
        -1.57515645e-01, -3.82688120e-02,  8.49375352e-02,
        -1.47832423e-01, -1.33699819e-01,  8.34693834e-02,
         1.15936019e-01, -1.11149931e-02,  4.32767272e-02,
         2.45651141e-01,  6.05502613e-02, -3.29418038e-03,
        -4.50992770e-02, -9.60326102e-03, -2.59289611e-02,
        -5.88608086e-02, -8.86575952e-02, -4.77095880e-02,
         5.39274774e-02,  1.44684866e-01, -6.49816990e-02,
         1.29184965e-02,  2.49326937e-02,  2.42505188e+01]],      dtype=float32)

In [69]:
tf.reverse(test_instance, axis = [-1])

<tf.Tensor: shape=(1, 30), dtype=float64, numpy=
array([[ 0.00000000e+00, -5.86163935e-02,  2.10998182e-01,
         4.23963954e-01, -2.01199298e-01, -3.70869344e-01,
        -3.41323962e-01, -4.60620924e-02,  2.22981820e+00,
        -6.44864027e-01,  1.54380629e+00,  1.85637688e-01,
         3.35236557e-01, -5.55324333e-01, -7.86015840e-01,
        -1.10646280e+00,  9.54298638e-01,  3.10971711e-01,
        -1.88035726e+00,  1.06300824e+00,  1.48722209e-01,
        -2.34533311e+00, -1.30099101e+00,  1.30168094e+00,
         5.89556504e-01,  3.12313283e+00,  2.08574736e+00,
         7.80683602e-01, -1.51030791e+00,  4.33300000e+04]])>

In [70]:
denormalizer(autoencoder.predict(normalizer(tf.reverse(test_instance, axis = [-1]))))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


Array([[ 5.91586602e+04, -4.50955182e-01,  1.37564048e-01,
         7.74269998e-01,  6.93160743e-02, -2.52150357e-01,
        -1.57515645e-01, -3.82688120e-02,  8.49375352e-02,
        -1.47832423e-01, -1.33699819e-01,  8.34693834e-02,
         1.15936019e-01, -1.11149931e-02,  4.32767272e-02,
         2.45651141e-01,  6.05502613e-02, -3.29418038e-03,
        -4.50992770e-02, -9.60326102e-03, -2.59289611e-02,
        -5.88608086e-02, -8.86575952e-02, -4.77095880e-02,
         5.39274774e-02,  1.44684866e-01, -6.49816990e-02,
         1.29184965e-02,  2.49326937e-02,  2.42505188e+01]],      dtype=float32)

---
Enhancement - Named Inputs
- Prevents mistakes from the order of instance values changing

In [47]:
def prep_batch(source):
    inputs = {}
    for col in var_numeric:
        inputs[col] = source[col] 
    return inputs, inputs

In [48]:
batch, batch = next(iter(train_read.map(prep_batch).batch(10).take(1)))

In [50]:
batch.keys()

dict_keys(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'])

In [51]:
batch['Time']

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([ 83533., 113822.,  39200.,  61781., 148437., 123042., 127907.,
         1888., 117742., 163935.])>

In [52]:
# Normalization
normalizer = keras.layers.Normalization(axis=-1, name='normalization')

# Adapt the normalizer 
numeric_feature_reader = train_read.prefetch(10).map(prep_batch).batch(1000)
normalizer.adapt(numeric_feature_reader.map(lambda x, _: tf.stack(list(x.values()), axis=-1))) 

# Denormalization using normalizer parameters
denormalizer = keras.layers.Normalization(axis=-1, name='denormalize', invert=True, mean=normalizer.mean, variance=normalizer.variance)

In [53]:
# Create separate input layers for each numeric feature
input_layers = {}
for col in var_numeric:
    input_layers[col] = keras.Input(shape=(1,), name=col)
autoencoder_input = keras.layers.concatenate(list(input_layers.values()))

In [54]:
#autoencoder_input = keras.Input(shape = (len(var_numeric),), name = "autoencoder_input")
normalized_input = normalizer(autoencoder_input)
encoder = keras.layers.Dense(16, activation='relu', name='enc_dense1')(normalized_input)
encoder = keras.layers.Dropout(0.2, name='enc_dropout1')(encoder)
encoder = keras.layers.Dense(8, activation='relu', name='enc_dense2')(encoder)
encoder = keras.layers.Dropout(0.2, name='enc_dropout2')(encoder)
encoder = keras.layers.Dense(4, activation='relu', name='enc_dense3')(encoder)
decoder = keras.layers.Dense(8, activation='relu', name='dec_dense1')(encoder)
decoder = keras.layers.Dropout(0.2, name='dec_dropout1')(decoder)
decoder = keras.layers.Dense(16, activation='relu', name='dec_dense2')(decoder)
decoder = keras.layers.Dropout(0.2, name='dec_dropout2')(decoder)
decoder = keras.layers.Dense(30, activation='linear', name='dec_dense3')(decoder)
reconstructed = denormalizer(decoder)

In [128]:
# Create separate output layers for each feature
output_layers = {}
for i, col in enumerate(var_numeric):
    output_layers[col] = keras.layers.Lambda(lambda x: x[:, i:i+1], name=f'output_{col}')(reconstructed)

You cannot directly pass KerasTensors (like normalized_input and decoder) to the keras.losses.MeanAbsoluteError() function when using the JAX backend. This is because keras.losses.MeanAbsoluteError() is a JAX function, and it expects JAX arrays as input.
```
# calculate loss between input and output of the autoencoder layers
mae_loss = keras.losses.MeanAbsoluteError()(normalized_input, decoder)
```

In [147]:
# Create a custom layer for calculating the combined MAE loss
class CombinedMAELossLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        normalized_input, decoder_output = inputs

        # Calculate the combined MAE loss
        mae_loss = tf.reduce_mean(tf.abs(normalized_input - decoder_output))

        # Add the combined MAE as a metric
        self.add_metric(mae_loss, name="combined_mae")

        # Add the combined MAE as the loss for this layer
        self.add_loss(mae_loss)

        return decoder_output  # Return a dummy output

# Calculate the combined MAE loss using the custom layer
combined_mae_loss = CombinedMAELossLayer(name="combined_mae_loss")(
    [normalized_input, decoder]
)

AttributeError: Exception encountered when calling CombinedMAELossLayer.call().

[1m'str' object has no attribute '_error_repr'[0m

Arguments received by CombinedMAELossLayer.call():
  • args=(['<KerasTensor shape=(None, 30), dtype=float32, sparse=False, name=keras_tensor_22>', '<KerasTensor shape=(None, 30), dtype=float32, sparse=False, name=keras_tensor_32>'],)
  • kwargs=<class 'inspect._empty'>

In [131]:
# Create a custom layer for calculating MAE loss
class MAELossLayer(keras.layers.Layer):
    def call(self, inputs):
        normalized_input, decoder_output = inputs
        mae_loss = keras.losses.MeanAbsoluteError()(normalized_input, decoder_output)
        return mae_loss
    
# Calculate MAE loss using the custom layer
mae_loss_layer = MAELossLayer()([normalized_input, decoder]) 

In [132]:
autoencoder = keras.Model(inputs = input_layers, outputs = [mae_loss_layer] + list(output_layers.values()), name = 'autoencoder')

In [133]:
autoencoder.compile(
    optimizer = keras.optimizers.Adam(),
    loss = [None] + ['mae'] * len(var_numeric)
)

In [119]:
autoencoder.compile(
    optimizer=keras.optimizers.Adam(),
    loss=lambda y_true, y_pred: y_pred,  # Use the predicted loss directly
    metrics=[
        [keras.metrics.MeanAbsoluteError(name='mae'), 
        keras.metrics.RootMeanSquaredError(name='rmse'),
        keras.metrics.MeanSquaredError(name='mse'),
        keras.metrics.MeanSquaredLogarithmicError(name='msle')],
        *([None] * 30)
    ]
)

In [134]:
autoencoder.summary()

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Time (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V4 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V5 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V6 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V7 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V8 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V9 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V10 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V11 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V12 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V13 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V14 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V15 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V16 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V17 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V18 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V19 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V20 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V21 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V22 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V23 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V24 (InputLayer)    │ (None, 1)         │          0 │ -               

 Total params: 1,423 (5.56 KB)

 Trainable params: 1,362 (5.32 KB)

 Non-trainable params: 61 (244.00 B)

In [136]:
# Data preparation for training
train_dataset = train_read.prefetch(10).map(prep_batch).batch(100)
val_dataset = validate_read.prefetch(10).map(prep_batch).batch(100)

In [137]:
history = autoencoder.fit(
    train_dataset,
    epochs = 10,
    validation_data = val_dataset
)

Epoch 1/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 46s 19ms/step - loss: 94192.8047 - mae_loss: 0.5091 - val_loss: 94562.8203 - val_mae_loss: 0.4938
Epoch 2/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 40s 17ms/step - loss: 94192.7812 - mae_loss: 0.5089 - val_loss: 94562.8281 - val_mae_loss: 0.4938
Epoch 3/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 39s 17ms/step - loss: 94192.7812 - mae_loss: 0.5088 - val_loss: 94562.8438 - val_mae_loss: 0.4938
Epoch 4/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 39s 17ms/step - loss: 94192.7891 - mae_loss: 0.5088 - val_loss: 94562.8438 - val_mae_loss: 0.4939
Epoch 5/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 39s 17ms/step - loss: 94192.7891 - mae_loss: 0.5088 - val_loss: 94562.8438 - val_mae_loss: 0.4938
Epoch 6/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 38s 17ms/step - loss: 94192.7812 - mae_loss: 0.5088 - val_loss: 94562.8438 - val_mae_loss: 0.4939
Epoch 7/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 40s 18ms/step - loss: 94192.7812 - mae_loss: 0.5088 - val_loss: 94562.8438 - val_mae_loss: 0.4938
Epoch 8/10
2279/2279

In [142]:
test_instance = next(iter(test_read.map(prep_batch).batch(1).take(1)))[0]#.numpy()
test_instance

{'Time': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([43330.])>,
 'V1': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-1.51030791])>,
 'V2': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.7806836])>,
 'V3': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([2.08574736])>,
 'V4': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([3.12313283])>,
 'V5': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.5895565])>,
 'V6': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([1.30168094])>,
 'V7': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-1.30099101])>,
 'V8': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-2.34533311])>,
 'V9': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.14872221])>,
 'V10': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([1.06300824])>,
 'V11': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-1.88035726])>,
 'V12': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.31097171])>,
 'V13': <tf.Tensor: shape=(1,), dty

In [143]:
autoencoder.predict(test_instance)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


ValueError: zero-dimensional arrays cannot be concatenated

In [58]:
autoencoder_input = keras.Input(shape = (len(var_numeric),), name = "autoencoder_input")
normalized_input = normalizer(autoencoder_input)
encoder = keras.layers.Dense(32, activation='relu', name='enc_dense1')(normalized_input)
encoder = keras.layers.Dropout(0.2, name='enc_dropout1')(encoder)
encoder = keras.layers.Dense(16, activation='relu', name='enc_dense2')(encoder)
encoder = keras.layers.Dropout(0.2, name='enc_dropout2')(encoder)
encoder = keras.layers.Dense(8, activation='relu', name='enc_dense3')(encoder)
encoded = encoder
decoder = keras.layers.Dense(16, activation='relu', name='dec_dense1')(encoder)
decoder = keras.layers.Dropout(0.2, name='dec_dropout1')(decoder)
decoder = keras.layers.Dense(32, activation='relu', name='dec_dense2')(decoder)
decoder = keras.layers.Dropout(0.2, name='dec_dropout2')(decoder)
decoder = keras.layers.Dense(30, activation='linear', name='dec_dense3')(decoder)
reconstructed = denormalizer(decoder)

autoencoder = keras.Model(
    inputs = autoencoder_input, 
    outputs = {'reconstructed': reconstructed},
    name = 'autoencoder'
)

autoencoder.compile(
    optimizer = keras.optimizers.Adam(),
    loss = {'reconstructed': keras.losses.MeanAbsoluteError()},
    metrics = {
        'reconstructed': [
            keras.metrics.RootMeanSquaredError(name='rmse'),
            keras.metrics.MeanSquaredError(name='mse'),
            keras.metrics.MeanAbsoluteError(name='mae'),
            keras.metrics.MeanSquaredLogarithmicError(name='msle')
        ]
    }
)

# Data preparation for training
train_dataset = train_read.prefetch(10).map(prep_batch).batch(100)
val_dataset = validate_read.prefetch(10).map(prep_batch).batch(100)

history = autoencoder.fit(
    train_dataset,
    epochs = 10,
    validation_data = val_dataset,
)

ValueError: In the dict argument `metrics`, key 'reconstructed' does not correspond to any model output. Received:
metrics={'reconstructed': [<RootMeanSquaredError name=rmse>, <MeanSquaredError name=mse>, <MeanAbsoluteError name=mae>, <MeanSquaredLogarithmicError name=msle>]}

In [40]:
instance = next(iter(test_read.map(prep_batch).batch(1).take(1)))[0].numpy()
instance

array([[ 4.33300000e+04, -1.51030791e+00,  7.80683602e-01,
         2.08574736e+00,  3.12313283e+00,  5.89556504e-01,
         1.30168094e+00, -1.30099101e+00, -2.34533311e+00,
         1.48722209e-01,  1.06300824e+00, -1.88035726e+00,
         3.10971711e-01,  9.54298638e-01, -1.10646280e+00,
        -7.86015840e-01, -5.55324333e-01,  3.35236557e-01,
         1.85637688e-01,  1.54380629e+00, -6.44864027e-01,
         2.22981820e+00, -4.60620924e-02, -3.41323962e-01,
        -3.70869344e-01, -2.01199298e-01,  4.23963954e-01,
         2.10998182e-01, -5.86163935e-02,  0.00000000e+00]])

In [41]:
autoencoder.predict(instance)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step


array([[ 4.1043676e+04, -4.3083081e-01,  3.4859994e-01,  9.0864396e-01,
         2.8234163e-01, -1.0169501e-01, -1.9621438e-01,  8.2337499e-02,
         5.1636048e-02, -5.3624310e-02, -1.8390632e-01,  2.1107332e-01,
         1.1533982e-01,  3.7093755e-02,  1.4556475e-01,  3.1027508e-01,
         9.8145328e-02, -3.0412519e-02, -1.2050614e-01, -4.4527762e-02,
        -2.6773306e-02, -1.2913945e-01, -2.2887911e-01, -4.8720278e-03,
         8.0836453e-02,  7.0976466e-02,  2.0507287e-02,  3.1646393e-02,
         2.8067801e-02,  1.0611104e+01]], dtype=float32)

Potential Improvements and Considerations:

Data Scaling: While you're using keras.layers.Normalization, consider other scaling methods like MinMaxScaler or StandardScaler from scikit-learn. Experiment to see which works best for your data.
Encoder/Decoder Depth and Width: The number of layers and units per layer in your encoder and decoder can be adjusted. Try different architectures to find the optimal balance between compression and reconstruction accuracy.
Activation Functions: While ReLU is common, you could experiment with other activation functions like sigmoid or tanh, especially in the decoder's output layer, depending on the range of your data.
Loss Function: You're using Mean Absolute Error, but other loss functions like Mean Squared Error might be suitable depending on your specific needs.
Regularization: Consider adding L1 or L2 regularization to your dense layers to prevent overfitting.
Hyperparameter Tuning: Experiment with different optimizers, learning rates, batch sizes, and dropout rates to optimize your model's performance.
Visualization: Use tools like TensorBoard to visualize your model's architecture, training progress, and metrics.
Explainability: If needed, consider techniques like SHAP (SHapley Additive exPlanations) to understand how your autoencoder is making predictions.